In [ ]:
import pandas as pd
import numpy as np
import folium
import ast

# Data Cleaning Functions

In [ ]:
# Removing duplicates from device observations

def clean_coordinates_df(df):
    return df.drop_duplicates(subset='device_id', keep='last')

In [ ]:
# Turning 'payload' from string to dictionary

def extract_path(df, col='payload', key='path', convert_string=True):
    if convert_string:
        df[col] = df[col].apply(lambda x: ast.literal_eval(x))
        return df[col].apply(lambda x: x.get(key))
    else:
        return df[col].apply(lambda x: x.get(key))

In [ ]:
# Getting the device path from the 'path'

def get_path(x, coordinates_df):
    path = []
    if x is not None:
        for i in x.split(','):
            try:
                path.append(coordinates_df[coordinates_df['device_id'] == i][['latitude', 'longitude']].values[0])
            except:
                path.append(coordinates_df[coordinates_df['device_id'] == i][['latitude', 'longitude']].values)
        return path
    else:
        return path

In [ ]:
# Getting gps location from android_debugging_messages

def android_gps_location(row):
    return list(ast.literal_eval(ast.literal_eval(row).get('gpsLocation')))

# Mapping Functions

In [ ]:
# Generates the map

def create_map(coordinates):
    return folium.Map(location=coordinates, zoom_start=16, tiles='cartodbpositron')

In [ ]:
# Plots the ducks on the map

def map_ducks(m, df, papa_id='44E855A4AE30'):    
    for i in range(0,len(df)):
        if df.iloc[i]['device_id'] == papa_id:
            folium.CircleMarker(location=[df.iloc[i]['latitude'], df.iloc[i]['longitude']],
                                radius=3,
                                color='blue',
                                fill=True,
                                fill_color='blue',
                                popup="<br>".join([str(df.iloc[i]['device_id']), 'PapaDuck'])
                               ).add_to(m)
        else:
            folium.CircleMarker(location=[df.iloc[i]['latitude'], df.iloc[i]['longitude']],
                                radius=2,
                                color='red',
                                popup="<br>".join([str(df.iloc[i]['device_id']), str(df.iloc[i]['device_type'])])
                               ).add_to(m)
    return m

In [ ]:
# Plots the path of the messages and saves it as html

def plot_path(gps_location, clusterdata, device_observations, papa_id='44E855A4AE30', arrow=True):
    #clusterdata = clusterdata.drop_duplicates(subset='path', keep='last')
    for idx,val in enumerate(clusterdata['path_coordinates']):
        test = folium.Map(location=gps_location, zoom_start=14, tiles='cartodbdark_matter')
        test = map_ducks(test, device_observations)
        folium.PolyLine(val, weight=1, color='green').add_to(test)
        if arrow:
            arr = []
            if len(val)>1:
                for i in range(len(val)-1):
                    if (len(val[i]) != 0 and len(val[i+1]) != 0):
                        arr.append(get_arrows(locations=[val[i], val[i+1]]))
                for tri in arr:
                    tri[0].add_to(test)
        for j in val:
            if len(j)==2:
                folium.CircleMarker(location=[j[0], j[1]],
                                    radius=1,
                                    color='green'
                                    ).add_to(test)
        test.save(outfile=str(idx)+'isabela.html')
    return print('Complete!')

In [ ]:
# https://medium.com/@bobhaffner/folium-lines-with-arrows-25a0fe88e4e

from collections import namedtuple

def get_arrows(locations, color='green', size=6, n_arrows=1):
    
    '''
    Get a list of correctly placed and rotated 
    arrows/markers to be plotted
    
    Parameters
    locations : list of lists of lat lons that represent the 
                start and end of the line. 
                eg [[41.1132, -96.1993],[41.3810, -95.8021]]
    arrow_color : default is 'blue'
    size : default is 6
    n_arrows : number of arrows to create.  default is 3
    Return
    list of arrows/markers
    '''

    Point = namedtuple('Point', field_names=['lat', 'lon'])

    # creating point from our Point named tuple
    p1 = Point(locations[i][0], locations[i][1])
    p2 = Point(locations[i+1][0], locations[i+1][1])

    # getting the rotation needed for our marker.  
    # Subtracting 90 to account for the marker's orientation
    # of due East(get_bearing returns North)
    rotation = get_bearing(p1, p2) - 90

    # get an evenly space list of lats and lons for our arrows
    # note that I'm discarding the first and last for aesthetics
    # as I'm using markers to denote the start and end
    arrow_lats = np.linspace(p1.lat, p2.lat, n_arrows + 2)[1:n_arrows+1]
    arrow_lons = np.linspace(p1.lon, p2.lon, n_arrows + 2)[1:n_arrows+1]

    arrows = []

    #creating each "arrow" and appending them to our arrows list
    for points in zip(arrow_lats, arrow_lons):
        arrows.append(folium.RegularPolygonMarker(location=points, 
                        fill_color=color, number_of_sides=3, 
                        radius=size, rotation=rotation))
    return arrows

def get_bearing(p1, p2):
    
    '''
    Returns compass bearing from p1 to p2
    
    Parameters
    p1 : namedtuple with lat lon
    p2 : namedtuple with lat lon
    
    Return
    compass bearing of type float
    
    Notes
    Based on https://gist.github.com/jeromer/2005586
    '''
    
    long_diff = np.radians(p2.lon - p1.lon)
    
    lat1 = np.radians(p1.lat)
    lat2 = np.radians(p2.lat)
    
    x = np.sin(long_diff) * np.cos(lat2)
    y = (np.cos(lat1) * np.sin(lat2) 
        - (np.sin(lat1) * np.cos(lat2) 
        * np.cos(long_diff)))
    bearing = np.degrees(np.arctan2(x, y))
    
    # adjusting for compass bearing
    if bearing < 0:
        return bearing + 360
    return bearing